In [1]:
using CSV, FileIO, DataFrames, Dates
using GMT

In [2]:
region="italy"
catalog="ingv"

df = CSV.read("./data/$region.csv", DataFrame);

In [9]:
df = df[df.Magnitude .>= 3.0,:];

In [10]:
min_lon = minimum(df.Longitude)
max_lon = maximum(df.Longitude)
min_lat = minimum(df.Latitude)
max_lat = maximum(df.Latitude);

In [11]:
italy_coords = (min_lon,max_lon,min_lat,max_lat)

(6.6172, 18.513, 35.501, 47.075)

In [5]:
coast(region=italy_coords, proj=:merc, land=:gray, rivers="a", shore="0.25p", show=true)

In [16]:
marker_size = [2^x/100 for x in df.Magnitude];
# C_markers_depth = makecpt(cmap="red,blue", range=(minimum(df.Depth),maximum(df.Depth)),continuous=true,);
# C_markers_depth = makecpt(cmap=:seis, range=(minimum(df.Depth),maximum(df.Depth)),continuous=true,);
C_markers_mag = makecpt(cmap=:seis, range=(minimum(df.Magnitude),maximum(df.Magnitude)),continuous=true, inverse=true);

makecpt [WARNING]: Without inc in -T option, -Z has no effect (ignored)


In [17]:
coast(region=italy_coords, 
        proj=:merc, land=:gray, rivers="a", shore="0.25p", alpha=50, 
        frame=(axes=:WSne,))
plot!(df.Longitude, df.Latitude, 
        markersize=marker_size, marker=:cc, markerline=:faint,
        cmap=C_markers_mag, zcolor=df.Magnitude, alpha=50, 
        show=true)

In [18]:
C_map = makecpt(cmap=:geo, range=(-8000,8000), continuous=true);
relief_map = grdcut("@earth_relief_30s", region=italy_coords);

makecpt [WARNING]: Without inc in -T option, -Z has no effect (ignored)
GMT [WARNING]: Remote dataset given to a data processing module but no registration was specified - default to gridline registration (if available)
grdcut [WARNING]: (w - x_min) must equal (NX + eps) * x_inc), where NX is an integer and |eps| <= 0.0001.
grdcut [WARNING]: w reset from 6.6172 to 6.61666666667
grdcut [WARNING]: (e - x_min) must equal (NX + eps) * x_inc), where NX is an integer and |eps| <= 0.0001.
grdcut [WARNING]: e reset from 18.513 to 18.5166666667
grdcut [WARNING]: (s - y_min) must equal (NY + eps) * y_inc), where NY is an integer and |eps| <= 0.0001.
grdcut [WARNING]: s reset from 35.501 to 35.5


In [105]:
grdview(relief_map, proj=:merc, figsize=10, surftype=(image=100,), 
        cmap=C_map, zsize=1.5,view=(180,90) ) # shade=(azimuth=100, norm="e0.8"),
# colorbar!(pos=(outside=:MR,), shade=0.4, xaxis=(annot=:auto,), ylabel=:m, show=true)#view=(135,45),
colorbar!(pos=(anchor=:TC,length=(12.5,0.6), horizontal=true, offset=(0,1.0)),
xaxis=(annot=:auto,),frame=(ylabel=:m, offset=(0,1.0)),view=(180,90),savefig="./results/$region/test_test.png", show=true)


In [87]:
basemap(region=italy_coords,frame=(axes=:WSne), proj=:merc)

grdview!(relief_map, proj=:merc, axis=:none, surftype=(image=100,), 
        cmap=C_map, zsize=1.5, alpha=40)

plot!(df.Longitude, df.Latitude, 
markersize=marker_size, marker=:cc, markerline=:faint,
cmap=C_markers_mag, zcolor=df.Magnitude, alpha=60,)

colorbar!(pos=(outside=:MR, offset=(1.0,0)), shade=0.4, xaxis=(annot=:auto,), frame=(xlabel=:Magnitude,),par=(MAP_LABEL_OFFSET=0.8,), show=true)

### Semi 3D View

In [91]:
basemap(region=italy_coords, frame=(axes=:SE), proj=:merc, view=(135,45))

grdview!(relief_map, proj=:merc, axis=:none, surftype=(image=100,), 
        cmap=C_map, zsize=1.5, alpha=40 , view=(135,45))

plot!(df.Longitude, df.Latitude, 
markersize=marker_size, marker=:cc, markerline=:faint,
cmap=C_markers_mag, zcolor=df.Magnitude, alpha=60,view=(135,45))

colorbar!(pos=(outside=:MR, offset=(1.3,0)), shade=0.4, xaxis=(annot=:auto,), frame=(xlabel="Magnitude",),par=(MAP_LABEL_OFFSET=0.8,), view=(135,45), show=true)

In [120]:
function scatter_2D(df, region, magnitude_threshold; magnitude=true, view=(180,90))

    df = df[df.Magnitude .>= magnitude_threshold,:];

    # Get region's coordinates
    min_lon = minimum(df.Longitude)
    max_lon = maximum(df.Longitude)
    min_lat = minimum(df.Latitude)
    max_lat = maximum(df.Latitude);

    # Create the map coordinates
    map_coords = (min_lon,max_lon,min_lat,max_lat)
    
    # Colormap for the region topography
    C_map = makecpt(cmap=:geo, range=(-8000,8000), continuous=true);
    # Relief map of the region
    relief_map = grdcut("@earth_relief_30s", region=map_coords);

    
    # control marker size based on magnitude
    marker_size = [2^x/100 for x in df.Magnitude];

    if magnitude == true
        # Marker colormap based on depth
        C_markers = makecpt(cmap=:seis, range=(minimum(df.Magnitude),maximum(df.Magnitude)),continuous=true,);
        colorbar_label = "Magnitude"
        zcolor_control = df.Magnitude
    
    else 
        # marker colorsize based on magnitude
        C_markers = makecpt(cmap=:seis, range=(minimum(df.Depth),maximum(df.Depth)),continuous=true, inverse=true);
        colorbar_label = "Depth"
        zcolor_control = df.Depth
    end


    basemap(region=map_coords,frame=(axes=:WSne), proj=:merc, view=view)

    grdview!(relief_map, proj=:merc, axis=:none, surftype=(image=1500,), 
            cmap=C_map, zsize=1.5, alpha=40 , view=view)

    plot!(df.Longitude, df.Latitude, 
            markersize=marker_size, marker=:cc, markerline=:faint,
            cmap=C_markers, zcolor=zcolor_control, alpha=60, view=view)

    colorbar!(pos=(outside=:MR, offset=(1.0,0)), shade=0.4, xaxis=(annot=:auto,), frame=(xlabel=colorbar_label,),par=(MAP_LABEL_OFFSET=0.8,), 
                view=view, savefig="./results/$region/$(region)_2D_mag_$(magnitude_threshold)_$(colorbar_label)_$(view).png", show=true)


end

scatter_2D (generic function with 2 methods)

In [121]:
scatter_2D(df, "italy", 3.0; magnitude=true, view=(145,45))
scatter_2D(df, "italy", 3.0; magnitude=false, view=(145,45))

scatter_2D(df, "italy", 3.0; magnitude=true, view=(180,90))
scatter_2D(df, "italy", 3.0; magnitude=false, view=(180,90))

makecpt [WARNING]: Without inc in -T option, -Z has no effect (ignored)
GMT [WARNING]: Remote dataset given to a data processing module but no registration was specified - default to gridline registration (if available)
grdcut [WARNING]: (w - x_min) must equal (NX + eps) * x_inc), where NX is an integer and |eps| <= 0.0001.
grdcut [WARNING]: w reset from 6.6172 to 6.61666666667
grdcut [WARNING]: (e - x_min) must equal (NX + eps) * x_inc), where NX is an integer and |eps| <= 0.0001.
grdcut [WARNING]: e reset from 18.513 to 18.5166666667
grdcut [WARNING]: (s - y_min) must equal (NY + eps) * y_inc), where NY is an integer and |eps| <= 0.0001.
grdcut [WARNING]: s reset from 35.501 to 35.5
makecpt [WARNING]: Without inc in -T option, -Z has no effect (ignored)


makecpt [WARNING]: Without inc in -T option, -Z has no effect (ignored)
GMT [WARNING]: Remote dataset given to a data processing module but no registration was specified - default to gridline registration (if available)
grdcut [WARNING]: (w - x_min) must equal (NX + eps) * x_inc), where NX is an integer and |eps| <= 0.0001.
grdcut [WARNING]: w reset from 6.6172 to 6.61666666667
grdcut [WARNING]: (e - x_min) must equal (NX + eps) * x_inc), where NX is an integer and |eps| <= 0.0001.
grdcut [WARNING]: e reset from 18.513 to 18.5166666667
grdcut [WARNING]: (s - y_min) must equal (NY + eps) * y_inc), where NY is an integer and |eps| <= 0.0001.
grdcut [WARNING]: s reset from 35.501 to 35.5
makecpt [WARNING]: Without inc in -T option, -Z has no effect (ignored)


makecpt [WARNING]: Without inc in -T option, -Z has no effect (ignored)
GMT [WARNING]: Remote dataset given to a data processing module but no registration was specified - default to gridline registration (if available)
grdcut [WARNING]: (w - x_min) must equal (NX + eps) * x_inc), where NX is an integer and |eps| <= 0.0001.
grdcut [WARNING]: w reset from 6.6172 to 6.61666666667
grdcut [WARNING]: (e - x_min) must equal (NX + eps) * x_inc), where NX is an integer and |eps| <= 0.0001.
grdcut [WARNING]: e reset from 18.513 to 18.5166666667
grdcut [WARNING]: (s - y_min) must equal (NY + eps) * y_inc), where NY is an integer and |eps| <= 0.0001.
grdcut [WARNING]: s reset from 35.501 to 35.5
makecpt [WARNING]: Without inc in -T option, -Z has no effect (ignored)


makecpt [WARNING]: Without inc in -T option, -Z has no effect (ignored)
GMT [WARNING]: Remote dataset given to a data processing module but no registration was specified - default to gridline registration (if available)
grdcut [WARNING]: (w - x_min) must equal (NX + eps) * x_inc), where NX is an integer and |eps| <= 0.0001.
grdcut [WARNING]: w reset from 6.6172 to 6.61666666667
grdcut [WARNING]: (e - x_min) must equal (NX + eps) * x_inc), where NX is an integer and |eps| <= 0.0001.
grdcut [WARNING]: e reset from 18.513 to 18.5166666667
grdcut [WARNING]: (s - y_min) must equal (NY + eps) * y_inc), where NY is an integer and |eps| <= 0.0001.
grdcut [WARNING]: s reset from 35.501 to 35.5
makecpt [WARNING]: Without inc in -T option, -Z has no effect (ignored)


In [74]:
# C_map = makecpt(cmap=:relief, range=(-8000,8000), continuous=true);
# C_map = makecpt(cmap=:world, range=(-7000,7000), continuous=true);
C_map = makecpt(cmap=:geo, range=(-8000,8000), continuous=true);
# C_map = makecpt(cmap=:earth, range=(-11000,9000), continuous=true);
# C_map = makecpt(cmap=:etopo1, range=(-11000,8500), continuous=true);
# C_map = makecpt(cmap=:globe, range=(-10000,10000), continuous=true);
relief_map = grdcut("@earth_relief_30s", region=italy_coords);

makecpt [WARNING]: Without inc in -T option, -Z has no effect (ignored)
GMT [WARNING]: Remote dataset given to a data processing module but no registration was specified - default to gridline registration (if available)
grdcut [WARNING]: (w - x_min) must equal (NX + eps) * x_inc), where NX is an integer and |eps| <= 0.0001.
grdcut [WARNING]: w reset from 6.6166 to 6.60833333333
grdcut [WARNING]: (e - x_min) must equal (NX + eps) * x_inc), where NX is an integer and |eps| <= 0.0001.
grdcut [WARNING]: e reset from 18.5166 to 18.5166666667
grdcut [WARNING]: (n - y_min) must equal (NY + eps) * y_inc), where NY is an integer and |eps| <= 0.0001.
grdcut [WARNING]: n reset from 47.0833 to 47.0833333333


In [75]:
grdview(relief_map, proj=:merc, figsize=10, surftype=(image=1000,), 
        view=(135,45),cmap=C_map, zsize=1.5, title="geo")
colorbar!(pos=(outside=:MR,), shade=0.4,view=(135,45), xaxis=(annot=:auto,), ylabel=:m, show=true)


In [ ]:
# C = makecpt(cmap=:dem2, range=(0,2000));
# G = grdcut("@earth_relief_30s", region=italy_coords);
Ggrad = grdgradient(G, norm=(laplace=0.8,), azim=100, colinfo=:g);
grdview(G, proj=:merc, figsize=10, surftype=(image=1000,), 
                shade=Ggrad, zsize=0.1, alpha=55)

plot!(df.Longitude, df.Latitude, 
markersize=marker_size, marker=:cc, markerline=:faint,
color=C, zcolor=df.Depth, alpha=50, 
show=true)

In [53]:
grdview(relief_map, proj=:merc, figsize=10, surftype=(image=3000,), 
        view=(135,35),cmap=C_map, zsize=1.5, alpha=40)


plot!(df.Longitude, df.Latitude, 
markersize=marker_size, marker=:cc, markerline=:faint,
color=C_markers, zcolor=df.Depth, alpha=60,
view=(135,35))

colorbar!(pos=(outside=:MR,), shade=0.4,view=(135,35), xaxis=(annot=:auto,), ylabel=:m, show=true)